
# prediccion de incidentes viales en la ciudad de Medellin

## importacion librerias necesarias

In [1]:
import pandas as pd
import re

## lectura y exploracion de los datos

In [2]:
#lectura dataframe saltando las filas malas
df=pd.read_csv("incidentes_viales.csv",on_bad_lines='skip',index_col=False,sep=";",encoding="UTF-8")
print(df.head())

    AÑO  CBML CLASE_ACCIDENTE      DIRECCION       DIRECCION ENCASILLADA  \
0  2015  1013  Caida Ocupante    CR 46 CL 43    CR  046   043  000 00000   
1  2015  1020          Choque  CR 43 A CL 29  CR  043 A   029  000 00000   
2  2015  0205            Otro    CR 46 CL 37    CR  046   037  000 00000   
3  2015  0101       Atropello   CR 34 CL 112    CR  034   112  000 00000   
4  2015  1504          Choque    CR 50 CL 14    CR  050   014  000 00000   

         DISEÑO  EXPEDIENTE      FECHA_ACCIDENTE          FECHA_ACCIDENTES  \
0  Tramo de via  A000259731  21/10/2015 05:58:00  2015-10-21T10:58:00.000Z   
1  Tramo de via  A000261725  05/11/2015 03:00:00  2015-11-05T08:00:00.000Z   
2  Tramo de via  A000259739  21/10/2015 07:40:00  2015-10-21T12:40:00.000Z   
3  Tramo de via  A000259752  21/10/2015 12:00:00  2015-10-21T17:00:00.000Z   
4  Tramo de via  A000259949  03/11/2015 06:16:00  2015-11-03T11:16:00.000Z   

  GRAVEDAD_ACCIDENTE  MES NRO_RADICADO NUMCOMUNA                     BARRI

C:\Users\johan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#revisamos las columnas
print(df.columns)
print("numero de columnas iniciales ",len(df.columns))
print("numero de filas iniciales ",df.shape[0])


Index(['AÑO', 'CBML', 'CLASE_ACCIDENTE', 'DIRECCION', 'DIRECCION ENCASILLADA',
       'DISEÑO', 'EXPEDIENTE', 'FECHA_ACCIDENTE', 'FECHA_ACCIDENTES',
       'GRAVEDAD_ACCIDENTE', 'MES', 'NRO_RADICADO', 'NUMCOMUNA', 'BARRIO',
       'COMUNA', 'LOCATION', 'X', 'Y'],
      dtype='object')
numero de columnas iniciales  18
numero de filas iniciales  270765


## limpieza de datos

In [4]:
#eliminacion de datos con nulos
df.dropna(inplace=True)
#eliminar datos duplicados
df.drop_duplicates(inplace=True)
print(df.shape[0])

250686


In [5]:
#AÑO-Año de ocurrencia del incidente
df["AÑO"]=df["AÑO"].apply(lambda x : re.findall(r"[0-9]{4}",str(x))[0])
print(df["AÑO"].unique())

['2015' '2014' '2016' '2017' '2018' '2019' '2020']


In [6]:
#CBML-Codigo catastral que corresponde al codigo comuna, barrio, manzana, lote catastral de un predio.
print(len(df["CBML"].unique()))

521


In [7]:
#CLASE_ACCIDENTE-Clasificación del IPAT sobre la clase de accidente de transito: choque, atropello, volcamiento,
#caida de ocupante, incendio, u otro (que no corresponde a las anteriores 5 clasificaciones, p. ej: sumersión)
df["CLASE_ACCIDENTE"]=df["CLASE_ACCIDENTE"].apply(lambda x : str(x).replace("Caída de Ocupante","Caida Ocupante").lower())
print((df["CLASE_ACCIDENTE"].unique()))

['caida ocupante' 'choque' 'otro' 'atropello' 'volcamiento' 'incendio']


In [8]:
#DIRECCION-Dirección donde ocurrió el incidente
##aparentemente se encuentra ya formateada
print((df["DIRECCION"].sample(10)))

25557         CR 48 CL 89
75926       CL 50 CR 56 B
131868        CL 35 CR 32
251491       CR 124 CL 55
112577        CL 48 CR 80
258404        CR 28 CL 69
144031        CL 47 CR 43
179491        CR 61 CL 57
152461        CL 98 CR 73
92543     DG 74 B CL 32 B
Name: DIRECCION, dtype: object


In [9]:
#DIRECCION ENCASILLADA-	Dirección encasillada que entrega el geocodificador
##aparentemente se encuentra ya formateada
##variable con poca informacion util dada la variable anterior--escoger una de las 2
print((df["DIRECCION ENCASILLADA"].sample(10)))

232429      CL  029   045  000 00000
93779       CL  037   046  000 00000
138997      CR  055   049  000 00000
181238      CR  050   008  000 00000
58896       CR  036   043  000 00000
237942      CL  072   040  000 00000
14616       CR  065   094  000 00000
99031       CR  049   010  000 00000
41138     CR  065 D   043  000 00000
263314      CR  074   048  000 00000
Name: DIRECCION ENCASILLADA, dtype: object


In [10]:
#DISENO-Sitio de la vía donde ocurrió el accidente: Cicloruta, Glorieta, Interseccion, Lote o Predio, Paso a Nivel, Paso Elevado,
#Paso Inferior, Pontón, Puente, Tramo de via, Tunel, Via peatonal
df["DISEÑO"]=df["DISEÑO"].apply(lambda x : str(x).replace("Pont\\xF3n","Pontón").lower())
print((df["DISEÑO"].unique()))

['tramo de via' 'interseccion' 'glorieta' 'lote o predio' 'paso a nivel'
 'puente' 'paso elevado' 'ciclo ruta' 'paso inferior' 'via peatonal'
 'pontón' 'tunel']


In [11]:
#EXPEDIENTE
#Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
##variable  aprarentemente con poco valor
print((df["EXPEDIENTE"].sample(10)))

102758    A000109845
79681     A000183550
219165    A000374115
269677    A001160997
50083     A000184194
267241    A001153525
165196    A001001560
33503     A000997860
22227     A000843188
231587    A000311862
Name: EXPEDIENTE, dtype: object


In [12]:
#FECHA_ACCIDENTE
#Fecha del accidente, proviene del IPAT - Informe Policial de accidente de Tránsito
df["FECHA_ACCIDENTE"] = pd.to_datetime(df["FECHA_ACCIDENTE"],dayfirst=True)
print((df["FECHA_ACCIDENTE"].sample(10)))

243082   2016-07-12 15:40:00
196784   2017-09-06 08:30:00
169947   2019-10-06 00:30:00
226835   2016-12-06 15:15:00
78152    2015-04-13 23:30:00
233376   2016-04-08 10:00:00
170305   2019-10-13 12:20:00
264162   2019-12-28 23:30:00
2672     2015-05-01 05:15:00
123506   2015-10-29 10:00:00
Name: FECHA_ACCIDENTE, dtype: datetime64[ns]


In [13]:
#FECHA_ACCIDENTES
#Fecha de los accidente (formato YYYY-MM-DD hh:mi:ss), proviene del IPAT - Informe Policial de accidentes de Tránsito
##veriable redundante con la anterior
print((df["FECHA_ACCIDENTES"].head(10)))

0    2015-10-21T10:58:00.000Z
1    2015-11-05T08:00:00.000Z
2    2015-10-21T12:40:00.000Z
3    2015-10-21T17:00:00.000Z
4    2015-11-03T11:16:00.000Z
5    2015-11-05T09:30:00.000Z
6    2015-10-18T09:40:00.000Z
7    2015-11-04T09:46:00.000Z
8    2015-10-21T12:00:00.000Z
9    2015-11-05T12:00:00.000Z
Name: FECHA_ACCIDENTES, dtype: object


In [14]:
#GRAVEDAD_ACCIDENTE
#Clasificación del IPAT - Informe Policial de Accidentes de Tránsito, sobre la gravedad del accidente, corresponde al resultado 
#más grave presentado en el accidente. Daños materiales "Sólo daños", accidente con heridos "Herido", accidente con muertos 
#"Muerto". No indica cantidad
df["GRAVEDAD_ACCIDENTE"]=df["GRAVEDAD_ACCIDENTE"].apply(lambda x : str(x).replace("Solo da\\xF1os","Solo daños").lower())
print((df["GRAVEDAD_ACCIDENTE"].unique()))

['con heridos' 'solo daños' 'con muertos']


In [15]:
#MES
#Mes de ocurrencia del incidente vial
print((df["MES"].unique()))

[10 11 12  9  4  3  5  6  7  8  1  2]


In [16]:
#NRO_RADICADO
#Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
#no tiene mucho valor aparentemente para nuestros fines
print((df["NRO_RADICADO"].head()))

0    1508668
1    1510621
2    1508691
3    1508694
4    1510634
Name: NRO_RADICADO, dtype: object


In [17]:
#NUMCOMUNA
#Numero de la comununa en la que ocurrio incidente vial
print(df["NUMCOMUNA"].unique())

['10' '02' '01' '15' '14' '11' '07' '06' '04' '09' '05' '80' '13' '08'
 '16' '03' '12' '70' '60' '90' 'In' 'SN' '0' '50' 'AU' '5' '7' '9' '4' '3'
 '1' '2' '8' '6' 'Sin Inf']


In [18]:
#BARRIO
print(len(df["BARRIO"].unique()))
#Barrio de ocurrencia del incidente vial
#ó por o
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).lower().replace("\\xf3","o").replace("ó","o"))
#é por e 
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
#á por a 
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
#ñ
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xf1","ñ"))
#\\xed í por i
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xed","i").replace("í","i"))
#\\xfa ú por u
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xfa","u").replace("ú","u"))
print((df["BARRIO"].unique()))

489
['barrio colon' 'san diego' 'la francia' 'santo domingo savio no. 1'
 'santa fe' 'la florida' 'los conquistadores' 'la alpujarra' 'villa nueva'
 'facultad de minas u. nacional' 'la esperanza' 'perpetuo socorro'
 'campo valdes no. 1' 'miraflores' 'oleoducto' 'terminal de transporte'
 'gerona' 'u.d. atanasio girardot' 'alejandro echavarria'
 'cabecera san antonio de prado' 'manila' 'los balsos no.2' 'moravia'
 'tenche' 'universidad nacional' 'los colores' 'caribe'
 'juan xxiii la quiebra' 'belalcazar' 'santa cruz' 'sucre' 'la candelaria'
 'el estadio' 'suramericana' 'la gloria' 'centro administrativo'
 'las estancias' 'aures no. 2' 'los angeles' 'castilla' 'la salle'
 'la america' 'picacho' 'bosques de san pablo' 'la aguacatala'
 'playon de los comuneros' 'estacion villa' 'la colina' 'popular'
 'villa del socorro' 'cucaracho' 'santa maria de los angeles'
 'el chagualo' 'la mansion' 'san benito' 'la castellana' 'jesus nazareno'
 'loreto' 'asomadera no. 1' 'el salvador' 'cristo rey' 'm

In [19]:
#COMUNA
#Denominación con la cual se identifica cada Comuna o Corregimiento. 01:Popular 02:Santa Cruz 03:Manrique 04:Aranjuez 05:Castilla 06:Doce de Octubre 07:Robledo 08:Villa Hermosa 09:Buenos Aires 10:La Candelaria 11:Laureles - Estadio 12:La América 13:San Javier 14:El Poblado 15:Guayabal 16:Belén 50:San Sebastián de Palmitas 60:San Cristobal 70:Altavista 80:San Antonio de Prado 90:Santa Elena 99:Toda la Ciudad
##variable redundante
#ó por o
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).lower().replace("\\xf3","o").replace("ó","o"))
#á por a 
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
#é por e 
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
print((df["COMUNA"].unique()))

['la candelaria' 'santa cruz' 'popular' 'guayabal' 'el poblado'
 'laureles estadio' 'robledo' 'doce de octubre' 'aranjuez' 'buenos aires'
 'castilla' 'corregimiento de san antonio de prado' 'san javier'
 'villa hermosa' 'belen' 'manrique' 'la america'
 'corregimiento de altavista' 'corregimiento de san cristobal'
 'corregimiento de santa elena' 'in' 'sn'
 'corregimiento de san sebastian de palmitas' '0' 'au']


In [20]:
#LOCATION
#Fuente de información con la cual se realizó la geocodificación
print((df["LOCATION"].head(10)))

0    [-75.5688011014, 6.24312304123]
1    [-75.5693883283, 6.22870030622]
2    [-75.5503911403, 6.29869502502]
3    [-75.5442195069, 6.30019383306]
4     [-75.578180998, 6.21699243701]
5      [-75.561118009, 6.2047775959]
6    [-75.5810144962, 6.24576066323]
7    [-75.5752597216, 6.24685341912]
8    [-75.5624178534, 6.25341454089]
9    [-75.5931430393, 6.27330476643]
Name: LOCATION, dtype: object


In [21]:
#X
#Coordenada X en metros del accidente, en sistema de coordenadas MAGNA Medellin Local
print(df["X"].head(10))

0    834949.69
1    834880.17
2    837004.94
3    837688.53
4    833903.11
5    835788.30
6    833598.49
7    834235.89
8    835659.53
9    832264.64
Name: X, dtype: float64


In [22]:
#Y
#Coordenada Y en metros del accidente, en sistema de coordenadas MAGNA Medellin Local
print(df["Y"].head(10))

0    1182357.34
1    1180762.02
2    1188499.17
3    1188663.05
4    1179469.60
5    1178113.02
6    1182652.97
7    1182772.04
8    1183493.83
9    1185703.90
Name: Y, dtype: float64


In [23]:
#Eliminar duplicados
df.drop_duplicates(inplace=True)
#borrar columnas con datos redudantes o poco aportantes
df.drop(labels=["DIRECCION ENCASILLADA","EXPEDIENTE","FECHA_ACCIDENTES","NRO_RADICADO","COMUNA"],axis=1,inplace=True)

In [25]:
df.to_csv("datosLimpios.csv")
